# 🚀 GEAK-Agent Tutorial
### Generating Optimized GPU Kernels with LLM Agents

---

This notebook demonstrates **GEAK-Agent** - a multi-agent LLM framework that automatically generates and optimizes [Triton](https://github.com/openai/triton) GPU kernels.

| Step | What Happens |
|:----:|-------------|
| 1️⃣ | **Setup** - Configure API keys |
| 2️⃣ | **Initialize** - Load kernels and create agent |
| 3️⃣ | **Run** - Generate and optimize kernels |
| 4️⃣ | **Results** - View generated code and metrics |

---
## 1️⃣ Setup

In [ ]:
"""
git clone https://github.com/AMD-AGI/GEAK-agent.git
cd GEAK-agent
python3 -m pip install -r requirements.txt
"""

In [ ]:
# Initialize environment
import os, sys

TUTORIAL_DIR = os.getcwd() if os.getcwd().endswith('tutorial') else os.path.join(os.getcwd(), 'tutorial')
if TUTORIAL_DIR not in sys.path:
    sys.path.insert(0, TUTORIAL_DIR)

# Now import and run setup
from utils import setup_environment, print_header
TUTORIAL_DIR, SRC_DIR, CORPUS_PATH, TutorialDataloader = setup_environment()
print_header('✓ GEAK-Agent Tutorial Ready')

In [ ]:
# ═══════════════════════════════════════════════════════════════
#                       🔑 API CONFIGURATION
# ═══════════════════════════════════════════════════════════════

USE_AMD_API = True                              # Set False for OpenAI
AMD_API_KEY = 'YOUR_AMD_API_KEY'                # Your AMD API key
OPENAI_API_KEY = 'YOUR_OPENAI_KEY'              # Your OpenAI key

# ───────────────────────────────────────────────────────────────
API_KEY = AMD_API_KEY if USE_AMD_API else OPENAI_API_KEY
MODEL_ID = 'GPT4o' if USE_AMD_API else 'gpt-4o'
print(f'✓ API: {"AMD" if USE_AMD_API else "OpenAI"} | Model: {MODEL_ID}')

---
## 2️⃣ Initialize Dataset & Agent

In [ ]:
from models.OpenAI import OpenAIModel, StandardOpenAIModel
from agents.GaAgent import GaAgent

# Load kernels
dataset = TutorialDataloader(
    kernel_names=['add_example.py', 'sin_computation.py'],
    corpus_path=CORPUS_PATH
)

# Initialize model & agent
model = OpenAIModel(api_key=API_KEY, model_id=MODEL_ID) if USE_AMD_API \
    else StandardOpenAIModel(api_key=API_KEY, model_id=MODEL_ID)

agent = GaAgent(model=model, dataset=dataset, corpus_path=CORPUS_PATH, descendant_num=1)

print(f'✓ Loaded {len(dataset)} kernels | Agent ready with {len(agent.memories)} tasks')

In [ ]:
# Preview kernel instructions (optional)
from utils import display_kernel_info
display_kernel_info(dataset.problem_states)

---
## 3️⃣ Run GEAK-Agent

Each iteration: **Generate** → **Test** → **Benchmark** → **Reflect**

> ⏱️ Takes a few minutes per iteration

In [ ]:
from utils import print_config

OUTPUT_DIR = os.path.join(TUTORIAL_DIR, 'outputs')
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ═══════════════════════════════════════════════════════════════
#                       ⚙️ CONFIGURATION
# ═══════════════════════════════════════════════════════════════
CONFIG = {
    'iteration_num': 3,       # Optimization iterations
    'temperature': 1.0,       # LLM creativity (0.0-2.0)
    'descendant_num': 1,      # Candidates per iteration
    'ancestor_num': 3,        # Reference solutions
    'gpu_id': 0,              # GPU device
    'target_gpu': 'MI325X',   # Target AMD GPU
}
print_config(CONFIG, '🚀 Running GEAK-Agent')

In [ ]:
# Run the agent
agent.run(
    output_path=os.path.join(OUTPUT_DIR, 'tutorial_results.jsonl'),
    multi_thread=False,
    **CONFIG,
    start_iter=0, descendant_debug=0, profiling=False, start_idx=0
)
print('\n✅ Complete!')

---
## 4️⃣ View Results

In [ ]:
from utils import load_results, display_results_summary, display_generated_code

results, iteration = load_results(OUTPUT_DIR)
display_results_summary(results, iteration)

In [ ]:
# View generated code
display_generated_code(results)

---
## 📚 Reference

| Metric | Meaning |
|--------|--------|
| **pass_call** | Code runs without errors |
| **pass_exe** | Output matches reference |
| **speedup** | `ref_time/gen_time` (>1 = faster) |

**Tips:** More iterations → better results • Higher temperature → more diversity

---
🎉 **Done!** See [GEAK-Agent](https://github.com/AMD-AGI/GEAK-agent) for more.